# Clean ZIMAS / zoning file

In [ ]:
import numpy as np
import pandas as pd
import geopandas as gpd
import intake
import utils

In [ ]:
catalog = intake.open_catalog("../catalogs/*.yml")

In [ ]:
# Default value of display.max_rows is 10 i.e. at max 10 rows will be printed.
# Set it None to display all rows in the dataframe
pd.set_option('display.max_rows', None)

In [ ]:
df = catalog.zoning.read()
df = df[['ZONE_CMPLT']].drop_duplicates()

## Parse zoning string

In [ ]:
def parse_zoning(row):
    try:
        z = utils.ZoningInfo(row.ZONE_CMPLT)
        return pd.Series([z.Q, z.T, z.zone_class, z.specific_plan, z.height_district, z.D, z.overlay, z.invalid_zone, z.invalid_height], 
                         index = ['Q', 'T', 'zone_class', 'specific_plan', 'height_district', 'D', 'overlay', 'invalid_zone', 'invalid_height'])
    except ValueError:
        return pd.Series(['failed', 'failed', 'failed', 'failed', 'failed', 'failed', 'failed', '', ''], 
                         index = ['Q', 'T', 'zone_class', 'specific_plan', 'height_district', 'D', 'overlay', 'invalid_zone', 'invalid_height'])

    
parsed = df.apply(parse_zoning, axis = 1)

df = pd.concat([df, parsed], axis = 1)

df.head()

def parse_zoning(row):
    try:
        z = utils.ZoningInfo(row.ZONE_CMPLT)
        return pd.Series([z.Q, z.T, z.zone_class, z.specific_plan, z.height_district, z.D, z.overlay, z.invalid_zone, z.invalid_height], 
                        index = ['Q', 'T', 'zone_class', 'specific_plan', 'height_district', 'D', 'overlay', 'invalid_zone', 'invalid_height'])
    except ValueError:
        try:
            z = utils.Reparse_NoHyphen(row.ZONE_CMPLT)
            return pd.Series([z.Q, z.T, z.zone_class, z.specific_plan, z.height_district, z.D, z.overlay, z.invalid_zone, z.invalid_height], 
                             index = ['Q', 'T', 'zone_class', 'specific_plan', 'height_district', 'D', 'overlay', 'invalid_zone', 'invalid_height'])  
        except ValueError:
            try: 
                z = utils.Reparse_OnlyZone(row.ZONE_CMPLT)
                return pd.Series([z.Q, z.T, z.zone_class, z.specific_plan, z.height_district, z.D, z.overlay, z.invalid_zone, z.invalid_height], 
                index = ['Q', 'T', 'zone_class', 'specific_plan', 'height_district', 'D', 'overlay', 'invalid_zone', 'invalid_height'])  
            
            except ValueError:
                return pd.Series(['failed', 'failed', 'failed', 'failed', 'failed', 'failed', 'failed', '', ''], 
                                index = ['Q', 'T', 'zone_class', 'specific_plan', 'height_district', 'D', 'overlay', 
                                         'invalid_zone', 'invalid_height'])

parsed = df.apply(parse_zoning, axis = 1)

df = pd.concat([df, parsed], axis = 1)

df.head()

def parse_zoning(row):
    try:
        z = utils.ZoningInfo(row.ZONE_CMPLT)
        return pd.Series([z.Q, z.T, z.zone_class, z.specific_plan, z.height_district, z.D, z.overlay, z.invalid_zone, z.invalid_height], 
                        index = ['Q', 'T', 'zone_class', 'specific_plan', 'height_district', 'D', 'overlay', 'invalid_zone', 'invalid_height'])
    except ValueError:
        try:
            z = utils.Reparse_OnlyZone(row.ZONE_CMPLT)
            return pd.Series([z.Q, z.T, z.zone_class, z.specific_plan, z.height_district, z.D, z.overlay, z.invalid_zone, z.invalid_height], 
                             index = ['Q', 'T', 'zone_class', 'specific_plan', 'height_district', 'D', 'overlay', 'invalid_zone', 'invalid_height'])  
        except ValueError:
            try: 
                z = utils.Reparse_NoHyphen(row.ZONE_CMPLT)
                return pd.Series([z.Q, z.T, z.zone_class, z.specific_plan, z.height_district, z.D, z.overlay, z.invalid_zone, z.invalid_height], 
                index = ['Q', 'T', 'zone_class', 'specific_plan', 'height_district', 'D', 'overlay', 'invalid_zone', 'invalid_height'])  
            
            except ValueError:
                return pd.Series(['failed', 'failed', 'failed', 'failed', 'failed', 'failed', 'failed', '', ''], 
                                index = ['Q', 'T', 'zone_class', 'specific_plan', 'height_district', 'D', 'overlay', 
                                         'invalid_zone', 'invalid_height'])

parsed = df.apply(parse_zoning, axis = 1)

df = pd.concat([df, parsed], axis = 1)

df.head()

In [ ]:
df[(df.zone_class == 'invalid') | (df.zone_class == 'failed') | (df.zone_class == "")][['ZONE_CMPLT']]

In [ ]:
df[df.ZONE_CMPLT == 'USC-3']

In [ ]:
print(f"Obs that failed to be parsed: {len(df[df.Q == 'failed'])}")
print(f"Obs that failed to be parsed, invalid, or blanks: {len(df[(df.zone_class == 'invalid') | (df.zone_class == 'failed') | (df.zone_class == '')])}")

In [ ]:
""" 
With hyphen, try this:
    ([0-9A-Z]+)*(\([A-Z]+\))*([-1-4DLVXS]+)*((?:-[A-Z]+)*)$
"""

## Zone Class

In [ ]:
# Check if valid zone classes are used
df['valid_zone'] = df.zone_class.isin(utils.VALID_ZONE_CLASS)

df.valid_zone.value_counts()

In [ ]:
print(f'# obs with invalid zones: {len(df[df.valid_zone == False])}')
print(f'# of unique invalid zones: {df[df.valid_zone == False].zone_class.nunique()}')

In [ ]:
# Look at what these invalid zones are
# Appears that you can have multiple zones? R3 + P? Is this correct?
# Or, is there a hierarchy, where least restrictive includes all uses of most restrictive, and that spans across residential, agricultural, etc.
df[df.zone_class == 'invalid'].invalid_zone.value_counts()

In [ ]:
def zone_class(row):
    """
    This function tags the 35 different zone classes based
    on the first letter
    """
    
    open_space_zones = ['OS']
    agricultural_zones = ['A1', 'A2']
    residential_zones = ['RA', 'RE', 'RS', 'R1', 'RU', 'RZ', 'RW1', 
                         'R2', 'RD', 'RMP', 'RW2', 'R3', 'RAS3', 'R4', 'RAS4', 'R5']
    commercial_zones = ['CR', 'C1', 'C1.5', 'C4' ,'C2', 'C5', 'CM']
    manufacturing_zones = ['MR1', 'M1', 'MR2', 'M2', 'M3']
    
    text = row.zone_class
    
    if any(zone in text for zone in open_space_zones):
        return 'open_space'
    if any(zone in text for zone in agricultural_zones):
        return 'agricultural'
    if any(zone in text for zone in residential_zones):
        return 'residential'
    if any(zone in text for zone in commercial_zones):
        return 'commercial'
    if any(zone in text for zone in manufacturing_zones):
        return 'manufacturing'
    if 'PF' in text:
        return 'public_facility'
    if (('P' in text) or ('PB' in text)) and ('PF' not in text):
        return 'parking'
    if 'SL' in text:
        return 'submerged_lands'
    else:
        return ''

df['zone_class_descrip'] = df.apply(zone_class, axis = 1)   

df.zone_class_descrip.value_counts()

## Height District

In [ ]:
df['valid_height'] = df.apply(lambda row: False if row.height_district in(['', 'invalid']) else True, axis = 1)

df.valid_height.value_counts()

In [ ]:
# Look at what these invalid heights are.
# 1VL, 2VL seem like they could be legitimate. What is EZ1VL?
df[df.valid_height == False].invalid_height.value_counts()

In [ ]:
print(f'# obs with invalid height districts: {len(df[df.valid_height == False])}')
print(f'# of unique invalid height districts: {df[df.valid_height == False].invalid_height.nunique()}')

## D Limit

In [ ]:
df.D.value_counts()

In [ ]:
df[df.D == True].height_district.value_counts()
# Checked, if there is height district followed by a D, those are all tagged as True

In [ ]:
# Will need to make sure it's not mixed-type at time of export. Cannot be boolean and string
# Leave for now, because the failed to be parsed obs are still here
#df.D = df.replace('', False)
#df.D = df.D.astype(bool)

## Overlay

In [ ]:
type(df.overlay.iloc[1])

In [ ]:
# Fill in Nones, otherwise cannot do the apply to make the list a string
just_overlay = df[df.overlay != 'failed'][['overlay']]

In [ ]:
just_overlay['no_brackets'] = just_overlay['overlay'].apply(', '.join)

In [ ]:
just_overlay.no_brackets.value_counts()

In [ ]:
# Make a dictionary for supplemental use (map to overlay)
supplemental_use_dict = {
    # Supplemental Use found in Table 2 or Zoning Code Article 3
    'O': 'oil_drilling', 
    'S': 'animal_slaughtering', 
    'G': 'surface_mining', 
    'K': 'equinekeeping', 
    'CA': 'commercial_and_artcraft', 
    'MU': 'mixed_use', 
    'FH': 'fence_heights', 
    'SN': 'sign', 
    'HS': 'hillside_standards',
    'RG': 'rear_detached_garage', 
    'RPD': 'residential_planned_development', 
    'POD': 'pedestrian_oriented_district',
    'CDO': 'community_design_overlay',
    'NSO': 'neighborhood_stabilization_ordinance',
    'RFA': 'residential_floor_area',
    'MPR': 'modified_parking_requirement',
    'RIO': 'river_improvement_overlay',
    'HCR': 'hillside_construction_regulation',
    'CPIO': 'community_plan_implementation_overlay',
    'CUGU': 'clean_up_green_up_overlay',
    'HPOZ': 'historic_preservation_overlay_zone', 
}

In [ ]:
specific_plan_dict = {
    # Are other zoning designations put in with zone class or overlay? HPOX appears as overlay, but a lot of other ones appear in the beginning with zone class.
    # Even more designations found in Zoning Code Article 2
    'CEC': 'convention_and_event_center_specific_plan',
    # Other Zoning Designations found in Zoning Code Article 2, Sec 12.04 Zones - Districts - Symbols.
    # Are these always in parantheses in the beginning, near the prefix??
    'CW': 'central_city_west_specific_plan', 
    'GM': 'glencoe_maxella_specific_plan', 
    'OX': 'oxford_triangle_specific_plan', 
    'PV': 'playa_vista_specific_plan', 
    'WC': 'warner_center_specific_plan', 
    'ADP': 'alameda_district_specific_plan', 
    'CCS': 'century_city_south_studio_zone', 
    'CSA': 'centers_study_area', 
    'PKM': 'park_mile_specific_plan',
    'LAX': 'los_angeles_airport_specific_plan', 
    #'HPOZ': 'historic_preservation_overlay_zone', 
    'LASED': 'la_sports_and_entertainment_specific_plan',
    'USC-1A': 'usc_university_park_campus_specific_plan_subarea_1a_zone',
    'USC-1B': 'usc_university_park_campus_specific_plan_subarea_1b_zone',    
    'USC-2': 'usc_university_park_campus_specific_plan_subarea_2_zone',
    'USC-3': 'usc_university_park_campus_specific_plan_subarea_3_zone',
    'PVSP': 'ponte_vista_at_san_pedro_specific_plan'
}

In [ ]:
split = just_overlay.no_brackets.str.split(',', expand = True).fillna('')
split.rename(columns = {0: 'o1', 1: 'o2', 2: 'o3'}, inplace = True)

In [ ]:
# Compile list of unique overlays, and see how well that matches with the dict
unique_overlays = pd.DataFrame(split.o1.append(split.o2).append(split.o3))

In [ ]:
unique_overlays.rename(columns = {0: 'overlay'}, inplace = True)
unique_overlays.overlay = unique_overlays.overlay.str.strip()
unique_overlays = unique_overlays.drop_duplicates()
unique_overlays = unique_overlays.reset_index().drop(columns = ['index'])

In [ ]:
unique_overlays['overlay_descrip'] = unique_overlays.overlay.map(supplemental_use_dict)

In [ ]:
unique_overlays[unique_overlays.overlay_descrip.isna()]

In [ ]:
# What is SP? Specific Plan?
# Also, can't find H

## Misclasified
* Manually fix?

In [ ]:
# This is not working as it should -- o2 and o3 aren't getting filled
for col in ['o1', 'o2', 'o3']:
    new_col = f'{col}_descrip'
    df[col] = df[col].str.strip()
    df[new_col] = df[col].map(supplemental_use_dict)
    df[new_col] = df[new_col].fillna('')

In [ ]:
df.head()

In [ ]:
split = pd.DataFrame(just_overlay['overlay'].values.tolist(), columns = ['o1','o2', 'o3'], index = just_overlay.index).fillna('')

tag_overlays = pd.concat([just_overlay, split], axis = 1)

tag_overlays.head()

In [ ]:
def find_overlays(row, overlays, op=all):
    if row.overlay is None:
        return False
    else:
        return op([o in row.overlay for o in overlays])

df[df.apply(lambda x: find_overlays(x, ['RIO', 'CDO'], op=any), axis=1)]

In [ ]:
# Need to clean up where o1 didn't map to anything, and o2 and o3 are filled. Make sure the stuff that isn't overlays is removed.
# Maybe do long first, then make wide again?

In [ ]:
# Address the Nones first before converting to int
""" 
Q: Qualified Classification are restrictions to ensure compatibility with surrounding property
T: Tentative Zone Classifications are City Council requirements for public improvements

for col in ['Q', 'T']:
    df[col] = df[col].astype(int)
"""

## Merge back together

In [ ]:
full = catalog.zoning.read()
full.crs = {'init':'epsg:2229'}

In [ ]:
df2 = pd.merge(full, df, on = 'ZONE_CMPLT', how = 'left', validate = 'm:1')